<!-- Assignment 3 - WS 2020 -->

# Convolutional Neural Networks (16 points)

This notebook contains the third assignment for the exercises in Deep Learning and Neural Nets 1.
It provides a skeleton, i.e. code with gaps, that will be filled out by you in different exercises.
All exercise descriptions are visually annotated by a vertical bar on the left and some extra indentation,
unless you already messed with your jupyter notebook configuration.
Any questions that are not part of the exercise statement do not need to be answered,
but should rather be interpreted as triggers to guide your thought process.

**Note**: The cells in the introductory part (before the first subtitle)
perform all necessary imports and provide utility function that should work without problems.
Please, do not alter this code or add extra import statements in your submission, unless it is explicitly requested!

<span style="color:#d95c4c">**IMPORTANT:**</span> Please, change the name of your submission file so that it contains your student ID!

In this assignment, the goal is to get familiar with **Convolutional Neural Networks**. Essentially, a CNN is a multi-layer perceptron that uses convolutional instead of fully connected layers. Since convolutions are known to be useful for image processing, CNNs have become a powerful tool for learning features from images. However, they have proven to beat alternative architectures in a variety of other domains.

In [1]:
import numpy as np

from nnumpy import Module
from nnumpy.utils import sig2col
from nnumpy.testing import gradient_check

## Convolution

The main difference of CNNs with the fully connected networks we tackled thus far, is the *convolution operation*. 

###### The Math

Mathematically, a (discrete) convolution operates on two functions, so that

$$(f * g)[n] = \sum_{k \in \mathbb{Z}} f[k] g[n - k].$$

For image processing, the discrete functions $f$ and $g$ and replaced by images. After all, an image can be considered a function of pixel indices to pixel intensities. Also, images have (at least) two dimensions: width and height. Therefore, if we represent images as matrices of pixel intensities, we can write the convolution of an image $\boldsymbol{X} \in \mathbb{R}^{H \times W}$ with a so-called *kernel* $\boldsymbol{K} \in \mathbb{R}^{R_1 \times R_2}$ as follows:

$$(\boldsymbol{K} * \boldsymbol{X})_{a,b} = \sum_{i=1}^{R_1} \sum_{j=1}^{R_2} k_{i,j} x_{a - i + 1,b - j + 1}.$$

Instead of using the actual convolution operation, convolutional layers are often implemented as the *cross-correlation* of kernel and image instead:

$$(\boldsymbol{K} \star \boldsymbol{X})_{a,b} = \sum_{i=1}^{R_1} \sum_{j=1}^{R_2} k_{i,j} x_{a + i - 1,b + j - 1}.$$

It might be useful to note that unlike the convolution, the cross-correlation is not commutative, i.e. $\boldsymbol{K} \star \boldsymbol{X} \neq \boldsymbol{X} \star \boldsymbol{K}$, whereas $\boldsymbol{K} * \boldsymbol{X} = \boldsymbol{X} * \boldsymbol{K}$.

### Exercise 1: Cross-correlation vs Convolution (3 Points)

Implementation-wise, there is little difference between cross-correlation and convolution. It is even quite straightforward to implement one, given an implementation of the other. To keep things simple, this exercise is limited to the one-dimensional variants of these operations (for now). How hard would it be to make your implementation of the convolution function commutative?

> Implement functions to compute the cross-correlations and convolutions of one-dimensional signals. Obviously, you should **not** use functions like `np.convolve` or `np.correlate`.

In [2]:
def cross_correlation1d(x, k):
    """
    Compute a one-dimensional cross-correlation.
    
    Parameters
    ----------
    x : (L, ) ndarray
        Input data for the cross-correlation.
    k : (R, ) ndarray
        Kernel weights for the cross-correlation.
        
    Returns
    -------
    features : (L') ndarray
        Cross-correlation of the input data with the kernel.
    """
    L = len(x)
    R = len(k)
    out_len = L - R + 1
    
    # my solution
    feature = np.empty(out_len)
    # correlation over vector
    for a in range(out_len):
        feature[a] = x[a:a + R] @ k
    
    # solution
    feature = [k @ x[a:a+len(k)] for a in range(out_len)]
    
    return feature
    raise NotImplementedError("TODO: implement cross_correlation1d function!")

def convolution1d(x, k):
    """
    Compute a one-dimensional convolution.
    
    Parameters
    ----------
    x : (L, ) ndarray
        Input data for the convolution.
    k : (R, ) ndarray
        Kernel weights for the convolution.
        
    Returns
    -------
    features : (L', ) ndarray
        Result of convolving the input data with the kernel.
    """
    # make reverse vector of kernel
    k_flip = np.flip(k) # k[::-1] 
    feature = cross_correlation1d(x, k_flip)
    return feature


def cummutative_convolution1d(x, k):
    """
    Compute a one-dimensional convolution.
    
    Parameters
    ----------
    x : (L, ) ndarray
        Input data for the convolution.
    k : (R, ) ndarray
        Kernel weights for the convolution.
        
    Returns
    -------
    features : (L', ) ndarray
        Result of convolving the input data with the kernel.
    """
    # swap the kernel and input if kernel is larger
    if len(k)> len(x):
        x, k = k, x

    return convolution1d(x, k)
    raise NotImplementedError("TODO: implement convolution1d function!")

In [3]:
# sanity check
x = np.random.randn(11)
k = np.random.randn(3)

corr = cross_correlation1d(x, k)
corr_check = np.allclose(corr, np.correlate(x, k, mode='valid'))
print("cross correlation check:" "passed" if corr_check else "failed")
conv = convolution1d(x, k)
conv_check = np.allclose(conv, np.convolve(x, k, mode='valid'))
print("convolution       check:" "passed" if conv_check else "failed")

cross correlation check:passed
convolution       check:passed


###### The Code

This direct implementation does not offer a lot of features. For starters, it does not provide functionality to process multiple samples at once. Furthermore, practical implementations of convolutional layers normally require support for *channels*. After all, it is common practice to create multiple feature maps from a single signal to compensate for the spatial reduction through pooling and strides. These features can be incorporated in the mathematical formulation as follows:
$$(\boldsymbol{K} \star \boldsymbol{X})_{n,c_\mathrm{out},a,b} = \sum_{c_\mathrm{in}=1}^{C_\mathrm{in}} \sum_{i=1}^{R_1} \sum_{j=1}^{R_2} k_{c_\mathrm{out},c_\mathrm{in},i,j} x_{n,c_\mathrm{in},a + i - 1,b + j - 1}.$$

++Of course this makes things a bit more complicated. It also introduces an extra loop over the number of input channels. In order to implement the above formula efficiently, we can use a trick that is commonly referred to as `im2col`. The idea of `im2col` is to represent the input tensor ($\in \mathbb{R}^{N \times C_\mathrm{in} \times A \times B}$) by a tensor in $\mathbb{R}^{N \times A' \times B' \times (C_\mathrm{in} \cdot R_1 \cdot R_2)}$ where each "column" holds the elements in the window of the convolution. This allows the convolution to be computed as a simple matrix product with the (reshaped) kernel matrix $\boldsymbol{K} \in \mathbb{R}^{C_\mathrm{out} \times (C_\mathrm{in} \cdot R_1 \cdot R_2)}$, i.e.

$$(\boldsymbol{K} \star \boldsymbol{X})_{n,c_\mathrm{out},a,b} = \sum_{r=1}^{C_\mathrm{in} \cdot R_1 \cdot R_2} x_{n,a,b,r} k_{r,c_\mathrm{out}}.$$

This trick is (efficiently) implemented in the `sig2col` function (slightly different name, since the function allows for modalities other than images). It takes **two inputs**: the signal to be convolved and the shape of the kernel as a tuple.

In [4]:
# sig2col on 1D signal
x = np.arange(7)
sig2col(x, (3, ))

array([[0, 1, 2],
       [1, 2, 3],
       [2, 3, 4],
       [3, 4, 5],
       [4, 5, 6]])

In [5]:
# image
im = np.arange(16).reshape(4, 4)
im

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [6]:
# 3x2 windows in image as column vectors
sig2col(im, (3, 2)).reshape(2, 3, -1)

array([[[ 0,  1,  4,  5,  8,  9],
        [ 1,  2,  5,  6,  9, 10],
        [ 2,  3,  6,  7, 10, 11]],

       [[ 4,  5,  8,  9, 12, 13],
        [ 5,  6,  9, 10, 13, 14],
        [ 6,  7, 10, 11, 14, 15]]])

### Exercise 2: Multi-channel Convolutions (3 Points)

Time to implement an actually practical convolution function that can handle multiple channels. Let us make it a 2D convolution at once.

 > Implement the `multi_channel_convolution2d` function below. You can use the `sig2col` function to implement the convolution by means of a dot product.
 
**Hint:** When using the `sig2col` function, you might need to fiddle with the order of dimensions of your numpy arrays to align everything properly.

In [7]:
def multi_channel_convolution2d(x, k):
    """
    Compute the multi-channel convolution of multiple samples.
    
    Parameters
    ----------
    x : (N, Ci, A, B)
    k : (Co, Ci, R1, R2)
    
    Returns
    -------
    y : (N, Co, A', B')
    
    See Also
    --------
    sig2col : can be used to convert (N, Ci, A, B) ndarray 
              to (N, Ci, A', B', R1, R2) ndarray.
    """
    # my solution
    N, Ci, A, B = np.shape(x)
    Co, Ci, R1, R2 = np.shape(k)  # R1, R2 = k.shape[-2:]
    A_prime = A-R1+1
    B_prime = B-R2+1
    x_view = sig2col(x, (R1, R2))  # sig2col(x, (Ci, R1, R2)) 
    x_col = x_view.transpose(0,2,3,1,4,5).reshape(N, A_prime, B_prime, -1)
    y_col = x_col @ k.transpose(1,2,3,0).reshape(-1, Co)
    y = y_col.transpose(0, 3, 1, 2)  # y_col.reshape(N, Co, A_prime, B_prime) #
    
    # tensordot solution
    x_view = sig2col(x, k.shape[-2:])  #
    y_col = np.tensordot(x_view, k, axes=([1,4,5],[1,2,3]))
    y = y_col.transpose(0,3,1,2)
    
    # einsum solution
    # x_view = sig2col(x, k.shape[-2:])  #
    # y = np.einsum('ncabij,ocij->noab', x_view, k)
    
    return y
    raise NotImplementedError("TODO: implement multi_channel_convolution2d function!")

In [8]:
# sanity check
x = np.random.randn(10, 1, 28, 28)
k = np.random.randn(5, 1, 3, 3)
multi_channel_convolution2d(x, k).shape

(10, 5, 26, 26)

###### The Module

The multi-channel convolution pretty much covers the forward pass for a typical convolutional layer. For the backward pass, we will need the gradients of this operations. In the case of the simple convolution from the first exercise, it can easily be derived that the gradients w.r.t. inputs and weights are again convolutions, since

$$\begin{aligned}
    \frac{\partial L}{\partial w_i} & = \sum_a \frac{\partial L}{\partial s_a} \frac{\partial s_a}{\partial w_i} = \sum_a \delta_a x_{i+a} \\
    \frac{\partial L}{\partial x_i} & = \sum_a \frac{\partial L}{\partial s_a} \frac{\partial s_a}{\partial x_i} = \sum_{a'} w_{a'} \delta_{i-a'},
\end{aligned}$$

where

$$\begin{aligned}
    \frac{\partial s_a}{\partial w_i} & = \frac{\partial}{\partial w_i} \left( \sum_r w_r x_{a+r} \right) = x_{a+i} \\
    \frac{\partial s_a}{\partial x_i} & = \frac{\partial}{\partial x_i} \left( \sum_r w_r x_{a+r} \right) = w_{i - a}.
\end{aligned}$$

Fortunately, this approach generalises to multi-channel convolutions. For the convolution of a 1D signal with $c_\mathrm{i}$ channels so that the output has $c_\mathrm{o}$ channels, it can be verified that

$$\begin{aligned}
    \frac{\partial L}{\partial w_{c_\mathrm{o}, c_\mathrm{i}, i}} & = \sum_a \frac{\partial L}{\partial s_{c_\mathrm{o},a}} \frac{\partial s_{c_\mathrm{o},a}}{\partial w_{c_\mathrm{o}, c_\mathrm{i}, i}} = \sum_a \delta_{c_\mathrm{o},a} x_{c_\mathrm{i},i+a} \\
    \frac{\partial L}{\partial x_{c_\mathrm{i}, i}} & = \sum_{c_\mathrm{o}} \sum_a \frac{\partial L}{\partial s_{c_\mathrm{o},a}} \frac{\partial s_{c_\mathrm{o},a}}{\partial x_{c_\mathrm{i}, i}} = \sum_{c_\mathrm{o}} \sum_{a'} w_{c_\mathrm{o}, c_\mathrm{i}, a'} \delta_{c_\mathrm{o}, i-a'},
\end{aligned}$$

where

$$\begin{aligned}
    \frac{\partial s_{c_\mathrm{o},a}}{\partial w_{c_\mathrm{o}, c, i}} & = \frac{\partial}{\partial w_{c_\mathrm{o}, c, i}} \left( \sum_{c_\mathrm{i}} \sum_r w_{c_\mathrm{o}, c_\mathrm{i}, r} x_{c_\mathrm{i},a+r} \right) = x_{c, a+i} \\
    \frac{\partial s_{c_1,a}}{\partial x_{c_2, i}} & = \frac{\partial}{\partial x_{c_2,i}} \left( \sum_{c_\mathrm{i}} \sum_r w_{c_\mathrm{o}, c_\mathrm{i}, r} x_{c_\mathrm{i}, a+r} \right) = w_{c_1, c_2, i - a}.
\end{aligned}$$

We can conclude that the gradients of multi-channel convolutions can again be expressed as multi-channel convolutions - taking into account that we compute the convolutions for multiple samples at once.

### Exercise 3: Convolutional Layer (5 Points)

Now, you should be able to implement both forward and backward pass in a module. Have you already thought about the shape of the bias parameter?

 > Implement the `Conv2D` module below. You can use the `multi_channel_convolution2d` function from the previous exercise to implement forward and backward pass.

In [9]:
class Conv2D(Module):
    """ Numpy DL implementation of a 2D convolutional layer. """
    
    def __init__(self, in_channels, out_channels, kernel_size, use_bias=True):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.use_bias = use_bias
        
        # register parameters 'w' and 'b' here (mind use_bias!)        
        self.w = self.register_parameter('w', np.empty((out_channels, in_channels, *kernel_size )))  #*kernel_size = kernel_size[0], kernel_size[1]
        if use_bias:
            self.b = self.register_parameter('b', np.empty(out_channels))  # (out_channels, 1, 1))
        # self.b = self.register_parameter('b', np.empty(out_channels if use_bias else 0))
        # raise NotImplementedError("TODO: register parameters in Conv2D.__init__!")
        self.reset_parameters()
        
    def reset_parameters(self):
        """ Reset the parameters to some random values. """
        self.w = np.random.randn(*self.w.shape)
        if self.use_bias:
            self.b = np.random.randn(*self.b.shape)
        
    def compute_outputs(self, x):
        """
        Parameters
        ----------
        x : (N, Ci, H, W) ndarray
        
        Returns
        -------
        feature_maps : (N, Co, H', W') ndarray
        cache : ndarray or tuple of ndarrays
        """
        feature_map = multi_channel_convolution2d(x, self.w)
        if self.use_bias:
            feature_map += self.b[None, :, None, None]
        cache = x
        return feature_map, cache
        raise NotImplementedError("TODO: implement Conv2D.compute_outputs function!")
    
    def compute_grads(self, grads, cache):
        """
        Parameters
        ----------
        grads : (N, Co, H', W') ndarray
        cache : ndarray or tuple of ndarrays
        
        Returns
        -------
        dx : (N, Ci, H, W) ndarray
        """
        x = cache
        if self.use_bias:
            self.b.grad = np.sum(grads, axis=(0, 2, 3))
            # solution 
            # total_grads = np.sum(grads, axis=0)
            # self.b.grad = np.sum(total_grads, axis=(-1,2), keepdims=True)
            
        # my solution
        # self.w.grad = multi_channel_convolution2d(x.transpose(1, 0, 2, 3), grads.transpose(1, 0, 2, 3)).transpose(1, 0, 2, 3)
        # solution
        self.w.grad = multi_channel_convolution2d(x.swapaxes(0,1), grads.swapaxes(0,1)).swapaxes(0,1)
        
        
        # my solution
        H_pad = self.kernel_size[0]-1  # x.shape[2] - grads.shape[2]
        W_pad = self.kernel_size[1]-1  # x.shape[3] - grads.shape[3]
        # grads_pad = np.zeros((grads.shape[0], grads.shape[1], grads.shape[2]+2*H_pad, grads.shape[3]+2*W_pad))
        # grads_pad[:, :, H_pad:-H_pad, W_pad:-W_pad] = grads
        grads_pad = np.pad(grads, ((0, 0), (0, 0), (H_pad, H_pad), (W_pad, W_pad)))
        
        # solution
        # padding = np.asarray(self.kernel_size) - 1
        # grads_pad = np.pad(grads, ((0,0),(0,0)) + tuple(zip(padding, padding)))
        
        # my solution
        # dx = multi_channel_convolution2d(grads_pad, self.w[:, :, ::-1, ::-1].transpose(1, 0, 2, 3)) 
        # solution 
        dx = multi_channel_convolution2d(grads_pad, self.w[:, :, ::-1, ::-1].swapaxes(0,1))
        return dx
        raise NotImplementedError("TODO: implement Conv2D.compute_grads function!")
        

In [10]:
# sanity check
conv2d = Conv2D(3, 8, (5, 3))
conv_check = gradient_check(conv2d, np.random.randn(15, 3, 13, 13), debug=True)
print("gradient check for Conv2D:", "passed" if conv_check else "failed")

gradient check for Conv2D: passed


## Activation Functions

Although any activation function can be used in combination with convolutional neural networks, a very popular choice is the so-called *Rectified Linear Unit* (*ReLU*). The ReLU function maps all negative inputs to zero and all positive inputs to itself. Mathematically, this looks like

$$\mathrm{ReLU}(x) = \begin{cases} 0 & x \leq 0 \\ x & x > 0 \end{cases}.$$

An alternative activation function that is based on the ReLU, is the *Exponential Linear Unit* (*ELU*). Unlike the ReLU non-linearity, the ELU is able to keep the mean of the activations close to zero. It can be defined as follows:

$$\mathrm{ELU}(x \mathbin{;} \alpha) = \begin{cases} \alpha (e^x - 1) & x \leq 0 \\ x & x > 0 \end{cases}.$$

The parameter $\alpha$ in this non-linearity allows to specify the minimal negative value of the activations. Note that this $\alpha$ is a hyper-parameter that must be fixed before training, and is thus not learned.

### Exercise 4: Some Linear Units (2 Points)

A deep learning framework would not be complete without the ReLU and ELU activation functions. Time to add them!

 > Implement the `ReLU` and `ELU` activation function modules.

In [11]:
class ReLU(Module):
    """ NNumpy implementation of the Rectified Linear Unit. """

    def compute_outputs(self, s):
        """
        Parameters
        ----------
        s : (N, K) ndarray

        Returns
        -------
        a : (N, K) ndarray
        cache : ndarray or iterable of ndarrays
        """

        a = np.maximum(0, s)
        cache = s
        return a, cache
        raise NotImplementedError("TODO: implement ReLU.compute_outputs method!")

    def compute_grads(self, grads, cache):
        """
        Parameters
        ----------
        grads : (N, K) ndarray
        cache : ndarray or iterable of ndarrays

        Returns
        -------
        ds : (N, K) ndarrays
        """
        s = cache
        # ds = np.zeros_like(s)
        # ds[s > 0] = 1
        # ds = grads * ds
        ds = np.where(s > 0, 1, 0) * grads
        return ds
        raise NotImplementedError("TODO: implement ReLU.compute_grads method!")


class ELU(Module):
    """ NNumpy implementation of the Exponential Linear Unit. """

    def __init__(self, alpha=1.):
        super().__init__()
        if alpha < 0:
            raise ValueError("negative values for alpha are not allowed")

        self.alpha = alpha

    def compute_outputs(self, s):
        """
        Parameters
        ----------
        s : (N, K) ndarray

        Returns
        -------
        a : (N, K) ndarray
        cache : ndarray or iterable of ndarrays
        """
        a = np.where(s > 0, s, self.alpha*(np.exp(s)-1))
        cache = s
        return a, cache
        raise NotImplementedError("TODO: implement ELU.compute_outputs method!")

    def compute_grads(self, grads, cache):
        """
        Parameters
        ----------
        grads : (N, K) ndarray
        cache : ndarray or iterable of ndarrays

        Returns
        -------
        ds : (N, K) ndarrays
        """
        s = cache
        ds = np.where(s > 0, 1, self.alpha * np.exp(s)) * grads
        return ds
        raise NotImplementedError("TODO: implement ELU.compute_grads method!")

In [12]:
# sanity check
relu_check = gradient_check(ReLU(), np.linspace(-3, 3), debug=True)
print("gradient check for ReLU:", "passed" if relu_check else "failed")
elu_check = gradient_check(ELU(1.5), np.linspace(-3, 3), debug=True)
print("gradient check for ELU: ", "passed" if elu_check else "failed")

gradient check for ReLU: passed
gradient check for ELU:  passed


## Spatial Reduction

The *weight sharing* in convolutional neural networks can drastically reduce the memory requirements for the weights. This effectively allows the input data to become larger, but since we need to store parts of the forward pass for back-propagation, the gains are rather limited. Of course, standard convolutions reduce the spatial dimensions, but this linear reduction is often too slow to counter the increased memory requirements due to network depth.

###### Pooling

In order to make working with big images feasible, we need techniques to reduce the spatial dimensions more strongly. This is where *pooling* layers prove very useful. A pooling layer reduces the spatial dimensions by combining a window of pixels to a single pixel. By sticking a pooling layer after every convolutional layer, the spatial dimensions are reduced exponentially, rather than linearly. This allows convolutional neural networks to process big chunks of data.

There are different ways to summarise multiple pixels into a single pixel. Two very common pooling techniques are

 1. **Average pooling** replaces the pixels by the mean intensity value in the window. 
 2. **Max pooling** replaces the pixels by the maximum intensity in the window.
 

###### Strides

In modern convolutional neural networks, *strided* or *dilated* convolutions (see visualisations below) are often preferred over pooling. With strided convolutions, the windows are shifted The main advantage of strided or dilated convolutions over pooling is that they can be learnt. This means that instead of relying on a fixed pooling technique, it is possible to effectively learn how the pixels in the window are to be summarised. Also note that average pooling can indeed be represented as a strided convolution with weights $\frac{1}{\text{window size}}$.

<div style="text-align: center">
  <figure style="display: inline-block; width: 49%;">
    <img style="padding: 46px 50px" src="https://raw.githubusercontent.com/vdumoulin/conv_arithmetic/master/gif/no_padding_strides.gif" />
    <figcaption style="width: 100%;"> Strided convolution </figcaption>
  </figure>
  <figure style="display: inline-block; width: 49%;">
    <img src="https://raw.githubusercontent.com/vdumoulin/conv_arithmetic/master/gif/dilation.gif" />
    <figcaption style="width: 100%; text-align: center;"> Dilated convolution </figcaption>
  </figure>
</div>

*visualisations taken from the [github](https://github.com/vdumoulin/conv_arithmetic) that comes with [this guide](https://arxiv.org/abs/1603.07285)*

### Exercise 5: Pooling (3 Points)

Since the `sig2col` function provides an array with the window elements in each column, it can also be used to implement pooling layers, when used correctly.

 > Implement the `AvgPool2d` module. You can use the `sig2col` function with its `stride` argument.
 
**Hint:** The gradient of a strided convolution corresponds to a transposed convolution with strides

In [35]:
class AvgPool2d(Module):
    """ Numpy DL implementation of a average pooling layer. """
    
    def __init__(self, kernel_size):
        super().__init__()
        self.kernel_size = tuple(kernel_size)
        self.kernel = np.ones((self.kernel_size[0], self.kernel_size[1]))/(self.kernel_size[0]*self.kernel_size[1]) # np.prod(self.kernel_size)

    def compute_outputs(self, x):
        """
        Parameters
        ----------
        x : (N, C, H, W) ndarray

        Returns
        -------
        a : (N, C, H', W') ndarray
        cache : ndarray or tuple of ndarrays
        """
        
        # my solution
        # N, C, H, W = x.shape
        # R1, R2 = self.kernel_size
        # x_col = sig2col(x, self.kernel_size, stride=self.kernel_size).reshape(N, C, H-R1+1, W-R2+1, -1)
        # a = x_col @ self.kernel.reshape(-1)
        # a = multi_channel_convolution2d(x, self.kernel[None, None, :, :])
        # cache = x
        
        # silly solution
        # s = x.reshape(*x.shape[:2], R1, x.shape[-2]//R1, R2, x.shape[-1]//R2)
        # a = np.mean(s, axis=(-2, -4))
        #  cache = np.array(x.shape)
        
        # solution 
        s = sig2col(x, self.kernel_size, stride=self.kernel_size)
        a = np.mean(s, axis=(-1, -2))
        cache = np.array(x.shape)
        
        return a, cache
        raise NotImplementedError("TODO: implement AvgPool2D.compute_outputs method!")

    def compute_grads(self, grads, cache):
        """
        Parameters
        ----------
        grads : (N, C, H', W') ndarray
        cache : ndarray or tuple of ndarrays

        Returns
        -------
        dx : (N, C, H, W) ndarray
        """
        # my solution (have problem)
        # x = cache
        # N, C, H_p, W_p = grads.shape
        # R1, R2 = self.kernel_size
        
        # padding to gards
        # grads_pad = np.zeros((N, C, H_p+2*(R1-1), W_p+2*(R2-1)))
        # grads_pad[:, :, (R1-1):-(R1-1), (R2-1):-(R2-1)] = grads
        # grads_pad = np.pad(grads, ((0, 0), (0, 0), (R1-1, R1-1), (R2-1, R2-1)))
        

        # grads_col = sig2col(grads_pad, self.kernel_size).reshape(N, C, H_p+R1-1, W_p+R2-1, -1)
        # dx = grads_col @ self.kernel.reshape(-1)
        ## dx = multi_channel_convolution2d(grads_pad, self.kernel[None, None, :, :])
        
        # solution
        padding = np.asarray(self.kernel_size) - 1
        padded_shape = tuple(cache[:-2]) + tuple(cache[-2:] + padding)
        _grads = np.zeros(padded_shape, dtype=grads.dtype)
        idx = (..., ) + tuple(slice(p, -p, k) for p, k in zip(padding, self.kernel_size))
        _grads[idx] = grads / np.prod(self.kernel_size)
        grads_col = sig2col(_grads, self.kernel_size)
        dx = np.sum(grads_col, axis=(-1, -2))

        return dx
        raise NotImplementedError("TODO: implement AvgPool2D.compute_grads method!")

In [36]:
# sanity check
pooling = AvgPool2d((2, 3))
pool_check = gradient_check(pooling, np.random.randn(1, 1, 16, 18), debug=True)
print("gradient check for AvgPool2D:", "passed" if pool_check else "failed")

gradient check for AvgPool2D: passed
